<a href="https://colab.research.google.com/github/Chy-kimtong/Fintech-Training-2022/blob/main/coinBase_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install coinbase

In [ ]:
from coinbase.wallet.client import Client
from coinbase.wallet.model import APIObject
import time
import pandas as pd

client = Client('VrKkI2ZsQOCJrwn4', '4aVjl7oj4fYzvNcxh6dVC96OHJiDiE1O')

list_price = []
stock_price = []
pos_held = False


#Get our information
account = client.get_primary_account()
asset = pd.DataFrame(account)
print(asset)
last_price_of_currency_pair = client.get_spot_price(currency_pair = 'BTC-USD')
print(last_price_of_currency_pair)

# get historical data of crypto within a specific time
price = client._make_api_object(client._get('v2', 'prices', 'BTC-USD', 'historic'), APIObject)
for i in price['prices']:
  list_price.append(float(i['price'])) #since the price is start from present to the past so first index is the current pric


#EMA Algorithm
while True:

  def calculate_ema(prices, days, smoothing = 2):
    ema = [sum(prices[:days]) / days]
    for price in prices[days:]:
        ema.append((price * (smoothing / (1 + days))) + ema[-1] * (1 - (smoothing / (1 + days))))
    return ema

  ema = calculate_ema(list_price, len(list_price))[0]
  last_price = last_price_of_currency_pair['amount']
  print("Exponential Moving Average: " + str(ema))
  print("Last Price: " + str(last_price))

  buy_price = client.get_buy_price(currency='USD')
  sell_price = client.get_sell_price(currency='USD')

  #Buy
  if float(buy_price['amount']) > ema and not pos_held: #If price is crossing EMA, and we haven't already bought -> so we buy it
    print("Buy")
    buy = client.buy(account_id = '55f3f6f0-8e8d-5487-bd83-7952cc9751c8',amount='1', currency="BTC")
    pos_held = True
    stock_price.append(last_price)
  print(stock_price)

  #Sell
  if len(stock_price) != 0:
    if float(sell_price['amount']) >= (stock_price[-1] + stock_price[-1] * 0.01) and pos_held:
      print("Sell take profit")
      sell = client.sell(account_id = '55f3f6f0-8e8d-5487-bd83-7952cc9751c8', amount='1', currency="BTC")
      pos_held = False
      stock_price.clear()
    elif float(sell_price['amount']) <= (stock_price[-1] - stock_price[-1] * 0.03) and pos_held:
      print("Sell stop loss")
      sell = client.sell(account_id = '55f3f6f0-8e8d-5487-bd83-7952cc9751c8', amount='1', currency="BTC")
      pos_held = False
      stock_price.clear()

    list_price.clear()
  time.sleep(10) #Refresh every 10 seconds



                                            id        name  primary    type  \
amount    55f3f6f0-8e8d-5487-bd83-7952cc9751c8  BTC Wallet     True  wallet   
currency  55f3f6f0-8e8d-5487-bd83-7952cc9751c8  BTC Wallet     True  wallet   

         currency     balance            created_at            updated_at  \
amount        BTC  0.00000000  2022-03-31T10:10:20Z  2022-04-01T03:25:06Z   
currency      BTC         BTC  2022-03-31T10:10:20Z  2022-04-01T03:25:06Z   

         resource                                      resource_path  \
amount    account  /v2/accounts/55f3f6f0-8e8d-5487-bd83-7952cc9751c8   
currency  account  /v2/accounts/55f3f6f0-8e8d-5487-bd83-7952cc9751c8   

          allow_deposits  allow_withdrawals native_balance  
amount              True               True           0.00  
currency            True               True            USD  
{
  "amount": "40590.65",
  "base": "BTC",
  "currency": "USD"
}
Exponential Moving Average: 40511.16561452512
Last Price: 40590.6

InvalidRequestError: ignored

In [ ]:
print(buy_price)
print(sell_price)


{
  "amount": "40816.46",
  "base": "BTC",
  "currency": "USD"
}
{
  "amount": "40399.38",
  "base": "BTC",
  "currency": "USD"
}
